# Chinese Remainder Theorem

Chinese Remainder Theorem (CRT) allows us to solve a system of congruences of the form:

$$ \begin{array}{rcll}
 x & \equiv & a_1 & (\mod m_1) \\
 x & \equiv & a_2 & (\mod m_2) \\
   & \vdots &     & \\
 x & \equiv & a_k & (\mod m_k) \\
 \end{array}$$
 

*NOTE:* $x$ is required to be a natural number.

There are two possible ways the problem could be posed: 

1. Find the *smallest natural number* $x$ that satisfies the congruences above.
2. Find (or somehow characterize) all natural numbers $x$ that satisfy the congruences above.


## Example: 
  
  Find $x$ such that 
  
  $$ \begin{array}{rcll}
  x & \equiv & 1 & (\mod 10) \\
  x & \equiv & 10 & (\mod 11) \\
  \end{array}$$
  
  Let us first examine a simple "enumeration" approach to solving the example.
  
  First let us look at all possible solutions to the first congruence:
  
  $$ x \equiv 1 (\mod 10) $$
  
  The natural numbers that can satisfy this equation are $x \in \{ 1, 11, \textbf{21}, 31, 41, 51, 61, \ldots \}$.
  
  Let us look at the second congruence:
  
  $$x \equiv 10 (\mod 11) $$
  
  The natural number  that can satisfy this equation are $x \in \{ 10, \textbf{21}, 32, 43, 54, 65, \ldots \}$.
  
By looking at the two sets, it is clear that the *smallest number $x$ that satisfies both congruences is $x = 21$*.

Next, what are the other values of $x$, if we went on with the enumeration, we will see that the values of $x$ that satisfy both congruences are 

$$ x \in \{ 21, 131, 241, 351, 461, 571, \ldots \}$$

A simple characterization is 

$$ x \equiv 21\ (\mod 110) $$.


Let us now revisit the system of congruences: 

$$ \begin{array}{rcll}
 x & \equiv & a_1 & (\mod m_1) \\
 x & \equiv & a_2 & (\mod m_2) \\
   & \vdots &     & \\
 x & \equiv & a_k & (\mod m_k) \\
 \end{array}$$
 
 Let $M$ be the number $m_1m_2 \cdots m_k$.
 
__Chinese Remainder Theorem (CRT):__ If $m_i, m_j$ are relatively prime for any pair $i, j$ such that $i \not= j$, then 

1. There is a smallest value of $x$ that satisfies the system of congruences above and 
2. For any $x$ that satisfies the system of congruences, the number $x + jM$ also satisfies the congruences for any natural number $j$.

Most importantly, it provides a means to calculate the solution through a simple algorithm



## Algorithm for Solving Congruences

0. First, let $M = m_1 m_2 \cdots m_k$.
1. Let $M_i$, for $i = 1, \ldots, k$ be given by the following formula: 
$$ M_i = \frac{m_1 m_2 \cdots m_k}{m_i} = \frac{M}{m_i}$$
2. For each $i = 1, \ldots, k$, calculate the modular inverse of $M_i$ modulo $m_i$. Let $y_i$ be the result.
$$ M_i y_i \equiv 1\ (\mod m_i)$$
3. Last, calculate the solution $x$ as 

$$ x = ( a_1M_1y_1 + a_2M_2y_2 + \cdots + a_kM_ky_k) \mod M $$

We can guarantee that $$x$$ is in fact the smallest solution that satisfies the congruences. 

Furthermore, every solution that satisfies the system of congruences can be generated by adding an integer multiple of $M$ to the solution $x$ obtained above.

## Step 1: Extended Euclid Algorithm

In [8]:
# First, let us code up extended euclid's algorithm
def extended_euclid(m, n):
    s1, t1 = 1, 0
    s2, t2 = 0, 1
    while n > 0:
        k = m % n
        q = m // n
        m, n = n, k
        s1, t1, s2, t2 = s2, t2, (s1 - q* s2), (t1 - q* t2)
    return m, s1, t1

In [9]:
g, s, t = extended_euclid(77, 14)
print('GCD of 77, 14=', g, 'Bezout coefficients s = ', s, 't=', t)

GCD of 77, 14= 7 Bezout coefficients s =  1 t= -5


In [10]:
g, s, t = extended_euclid(89, 114)
print('GCD of 89, 114=', g, 'Bezout coefficients s = ', s, 't=', t)

GCD of 89, 114= 1 Bezout coefficients s =  41 t= -32


## Step 2: Modular Inverse Computation

In [11]:
def compute_modular_inverse(a, m):
    a = a % m # Might as well compute a mod m
    g, s, t = extended_euclid(a, m)
    if g != 1: 
        print('ERROR: ', a, ',', m, ' are not relatively prime.')
    return s % m

In [17]:
b = compute_modular_inverse(4, 11)
print('Modular inverse of 4 (modulo 11) = ', b)
print(b * 4 % 11)

Modular inverse of 4 (modulo 11) =  3
1


In [16]:
b = compute_modular_inverse(135, 11)
print('Modular inverse of 135 (modulo 11) = ', b)
print(b * 135 %11)

Modular inverse of 135 (modulo 11) =  4
1


In [19]:
b = compute_modular_inverse(21878173, 31)
print('Modular inverse of  21878173 (modulo 31) = ', b)
print(b * 21878173 %31)

Modular inverse of  21878173 (modulo 31) =  2
1


## Step 3: Chinese Remainder Theorem: Calculate Solution

In [26]:
from functools import reduce

def solve_congruences_crt(lst_of_a_values, lst_of_m_values):
    assert len(lst_of_a_values) == len(lst_of_m_values) # BOTH lists must be of same size
    M = reduce(lambda x, y: x * y, lst_of_m_values) # multiply the m values together
    lst_of_M_i_values = [M//m for m in lst_of_m_values] # compute M_is
    # Compute y_i as the modular inverse of M_i modulo m_i
    # Elegant functional way to do it is to apply compute_modular_inverse by zipping up the two lists
    lst_of_y_i_values = [compute_modular_inverse(e[0], e[1]) for e in zip(lst_of_M_i_values, lst_of_m_values)]
    # Multiply a_i * M_i * y_i
    lst_of_a_M_y_values = [e[0]*e[1]*e[2] for e in zip(lst_of_a_values, lst_of_M_i_values, lst_of_y_i_values)]
    # Sum up and modulo M
    x = sum(lst_of_a_M_y_values) % M
    return x

In [27]:
# Solve the example: x == 1 (mod 10), x == 10 (mod 11)
x = solve_congruences_crt([1, 10], [10, 11])
print(x)

21


## Example 2: Solve the congruence

$$ \begin{array}{rcll}
x & \equiv & 10 & (\mod 15) \\
x & \equiv & 2 & (\mod 16) \\
x & \equiv & 31 & (\mod 49) \\
x & \equiv & 15 & (\mod 121) \\
\end{array}$$

In [32]:
x = solve_congruences_crt([10, 2, 31, 15], [15, 16, 49, 121])
print('x = ', x)
print('Verify the solution')
print (x % 15)
print (x % 16)
print (x % 49)
print (x % 121)

x =  696370
Verify the solution
10
2
31
15


## Example 3: Solve the Congruence

$$ \begin{array}{rcll}
x & \equiv & 15 & (\mod 16) \\
x & \equiv & 16 & (\mod 17) \\
x & \equiv & 30 & (\mod 31) \\
x & \equiv & 36 & (\mod 37) \\
\end{array}$$

In [34]:
x = solve_congruences_crt([15, 16, 30, 36], [16, 17, 31, 37])
print('x=', x)
print('Verify the solution')
print(x % 16)
print(x % 17)
print(x % 31)
print(x % 37)

x= 311983
Verify the solution
15
16
30
36
